# Additive Sharing Tensor

## Import libs

In [1]:
import syft as sy
import torch

import sympc

## Create the clients

In [2]:
alice = sy.VirtualMachine(name="alice")
bob = sy.VirtualMachine(name="bob")

alice_client = alice.get_client()
bob_client = bob.get_client()

## Create a session

The session is used to send some config information only once between the parties.
This information can be:
* the ring size in which we do the computation
* the precision and base
* the approximation methods we are using for different functions (TODO)

In [3]:
session = sy.lib.sympc.session.SySession(parties=[alice_client, bob_client])
print(session)

## Send the session to all the parties

In [4]:
session.setup_mpc()

In [5]:
x = sympc.tensor.AdditiveSharingTensor(secret=torch.tensor([1,2,3,4]), session=session)

In [6]:
print(x)

[AdditiveSharingTensor]
	[FixedPointEncoder]: precision: 4, base: 10
	<VirtualMachineClient: alice Client> -> TensorPointer
	<VirtualMachineClient: bob Client> -> TensorPointer


In [7]:
# The values does not make sense for any user, unless they "reconstruct" the original tensor
for val in x.shares:
    print(val.get()) ## Attention! get() destroys the pointer

tensor([-805832661310137163, 1637625446312185508, -989314220394355598,
         367429296782212909])
tensor([  805832661310147163, -1637625446312165508,   989314220394385598,
         -367429296782172909])


In [8]:
x_secret = torch.tensor([1,2,3,4])
x = sympc.tensor.AdditiveSharingTensor(secret=x_secret, session=session)

In [9]:
print(x.reconstruct()) ## Attention! Also reconstruct, destroys the pointers

tensor([1., 2., 3., 4.])


## Public Operations

In [10]:
x_secret = torch.tensor([1,2,3,4])
x = sympc.tensor.AdditiveSharingTensor(secret=x_secret, session=session)

y = 50

In [11]:
print("[AST + Pub] X + Y =", (x + y).reconstruct())
print("[Pub + Pub] X + Y =", x_secret + y)

[AST + Pub] X + Y = tensor([51., 52., 53., 54.])
[Pub + Pub] X + Y = tensor([51, 52, 53, 54])


In [12]:
print("[AST - Pub] X - Y =", (x - y).reconstruct())
print("[Pub - Pub] X - Y =", x_secret -y)

[AST - Pub] X - Y = tensor([-49., -48., -47., -46.])
[Pub - Pub] X - Y = tensor([-49, -48, -47, -46])


In [13]:
print("[AST * Pub] X * Y =", (x * y).reconstruct())
print("[Pub * Pub] X * Y =", x_secret * y)

[AST * Pub] X * Y = tensor([ 50., 100., 150., 200.])
[Pub * Pub] X * Y = tensor([ 50, 100, 150, 200])


## Private Operations

In [14]:
x_secret = torch.tensor([1, 2, 3])
x = sympc.tensor.AdditiveSharingTensor(secret=x_secret,session=session)

y_secret = 50
y = sympc.tensor.AdditiveSharingTensor(secret=y_secret, session=session)

In [15]:
print("[AST + AST] X + Y =", (x + y).reconstruct())
print("[Pub + Pub] X + Y =", x_secret + y_secret)

[AST + AST] X + Y = tensor([51., 52., 53.])
[Pub + Pub] X + Y = tensor([51, 52, 53])


In [16]:
print("[AST - AST] X - Y =", (x - y).reconstruct())
print("[Pub - Pub] X - Y =", x_secret - y_secret)

[AST - AST] X - Y = tensor([-49., -48., -47.])
[Pub - Pub] X - Y = tensor([-49, -48, -47])


In [17]:
print("[AST * AST] X * Y =", (x * y).reconstruct())
print("[Pub * Pub] X * Y =", x_secret * y_secret)

[AST * AST] X * Y = tensor([ 50., 100., 150.])
[Pub * Pub] X * Y = tensor([ 50, 100, 150])
